In [2]:
# text = "子供は公園で遊んだり、家で勉強したりします。"
text = "食べさせられました。この車は速くありませんでした。怒ってしまった。放り込んでおいてくれてよかった。"

In [3]:
from tabulate import tabulate

# SudachiPy

In [5]:
from sudachipy import dictionary, tokenizer
import pandas as pd

tokenizer_obj = dictionary.Dictionary(dict="full").create()
mode = tokenizer.Tokenizer.SplitMode.C

morphemes = tokenizer_obj.tokenize(text, mode)

data = []
for m in morphemes:
    data.append({
        "surface": m.surface(),
        "dictionary_form": m.dictionary_form(),
        "normalized_form": m.normalized_form(),
        "part_of_speech": m.part_of_speech(),
        "reading_form": m.reading_form(),
        "begin": m.begin(),
        "end": m.end(),
        "is_oov": m.is_oov(),
        "dictionary_id": m.dictionary_id(),
        "part_of_speech_id": m.part_of_speech_id(),
        "word_id": m.word_id(),
        "synonym_group_ids": m.synonym_group_ids(),
        "raw_surface": m.raw_surface()
    })

# print(tabulate(pd.DataFrame(data), headers="keys", tablefmt="github", showindex=False))
pd.DataFrame(data)

,surface,dictionary_form,normalized_form,part_of_speech,reading_form,begin,end,is_oov,dictionary_id,part_of_speech_id,word_id,synonym_group_ids,raw_surface
0,食べ,食べる,食べる,"(動詞, 一般, *, *, 下一段-バ行, 連用形-一般)",タベ,0,2,False,0,379,750623,[],食べ
1,させ,させる,させる,"(助動詞, *, *, *, 下一段-サ行, 未然形-一般)",サセ,2,4,False,0,831,65500,[],させ
2,られ,られる,られる,"(助動詞, *, *, *, 助動詞-レル, 連用形-一般)",ラレ,4,6,False,0,1433,168089,[],られ
3,まし,ます,ます,"(助動詞, *, *, *, 助動詞-マス, 連用形-一般)",マシ,6,8,False,0,1318,148276,[],まし
4,た,た,た,"(助動詞, *, *, *, 助動詞-タ, 終止形-一般)",タ,8,9,False,0,989,83559,[],た
5,。,。,。,"(補助記号, 句点, *, *, *, *)",。,9,10,False,0,1,6909,[],。
6,この,この,此の,"(連体詞, *, *, *, *, *)",コノ,10,12,False,0,61,60093,[],この
7,車,車,車,"(名詞, 普通名詞, 一般, *, *, *)",クルマ,12,13,False,0,4,704437,[13614],車
8,は,は,は,"(助詞, 係助詞, *, *, *, *)",ハ,13,14,False,0,799,121598,[],は
9,速く,速い,速い,"(形容詞, 一般, *, *, 形容詞, 連用形-一般)",ハヤク,14,16,False,0,75,716203,[404],速く


# Stanza

In [6]:
import pandas as pd
import stanza
from nltk import Tree

nlp = stanza.Pipeline(lang="ja", processors='tokenize, lemma, pos, constituency', model_dir=f"../toy_content/_stanza_resources", logging_level='WARN')

doc = nlp(text)

for sentence in doc.sentences:
    tokens = [word.to_dict() for word in sentence.tokens]
    print(sum(tokens, []))

    display(pd.DataFrame(sum(tokens, [])))
    # print(tabulate(pd.DataFrame(sum(tokens, [])), headers="keys", tablefmt="github", showindex=False))

    tree = Tree.fromstring(str(sentence.constituency))
    tree.pretty_print()   # Pretty text-based tree


/Users/chaosarium/Documents/Repos/Influx/influx_nlp/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/chaosarium/Documents/Repos/Influx/influx_nlp/.venv/lib/python3.12/site-packages/stanza/models/constituency/parse_transitions.py:322: SyntaxWarning: invalid escape sequence '\T'
  return "\Tree [.%s ? ]" % self.label


[{'id': 1, 'text': '食べ', 'lemma': '食べる', 'upos': 'VERB', 'xpos': '動詞-一般-下一段-バ行', 'start_char': 0, 'end_char': 2, 'misc': 'SpaceAfter=No'}, {'id': 2, 'text': 'さ', 'lemma': '為る', 'upos': 'AUX', 'xpos': '動詞-非自立可能-サ行変格', 'start_char': 2, 'end_char': 3, 'misc': 'SpaceAfter=No'}, {'id': 3, 'text': 'せ', 'lemma': 'せる', 'upos': 'AUX', 'xpos': '助動詞-下一段-サ行', 'start_char': 3, 'end_char': 4, 'misc': 'SpaceAfter=No'}, {'id': 4, 'text': 'られ', 'lemma': 'られる', 'upos': 'AUX', 'xpos': '助動詞-助動詞-レル', 'start_char': 4, 'end_char': 6, 'misc': 'SpaceAfter=No'}, {'id': 5, 'text': 'まし', 'lemma': 'ます', 'upos': 'AUX', 'xpos': '助動詞-助動詞-マス', 'start_char': 6, 'end_char': 8, 'misc': 'SpaceAfter=No'}, {'id': 6, 'text': 'た', 'lemma': 'た', 'upos': 'AUX', 'xpos': '助動詞-助動詞-タ', 'start_char': 8, 'end_char': 9, 'misc': 'SpaceAfter=No'}, {'id': 7, 'text': '。', 'lemma': '。', 'upos': 'PUNCT', 'xpos': '補助記号-句点', 'start_char': 9, 'end_char': 10, 'misc': 'SpaceAfter=No'}]


,id,text,lemma,upos,xpos,start_char,end_char,misc
0,1,食べ,食べる,VERB,動詞-一般-下一段-バ行,0,2,SpaceAfter=No
1,2,さ,為る,AUX,動詞-非自立可能-サ行変格,2,3,SpaceAfter=No
2,3,せ,せる,AUX,助動詞-下一段-サ行,3,4,SpaceAfter=No
3,4,られ,られる,AUX,助動詞-助動詞-レル,4,6,SpaceAfter=No
4,5,まし,ます,AUX,助動詞-助動詞-マス,6,8,SpaceAfter=No
5,6,た,た,AUX,助動詞-助動詞-タ,8,9,SpaceAfter=No
6,7,。,。,PUNCT,補助記号-句点,9,10,SpaceAfter=No


                     ROOT                  
                      |                     
                      S                    
                   ___|_________________    
                  S                     |  
                  |                     |   
                  VP                    |  
               ___|____________         |   
              VP               |        |  
           ___|_______         |        |   
          VP          |        VP       |  
       ___|_______    |     ___|___     |   
      VP          VP  VP   |       VP   |  
  ____|___        |   |    |       |    |   
VERB     AUX     AUX AUX  AUX     AUX PUNCT
 |        |       |   |    |       |    |   
 食べ       さ       せ   られ   まし      た    。  

[{'id': 1, 'text': 'この', 'lemma': '此の', 'upos': 'DET', 'xpos': '連体詞', 'start_char': 10, 'end_char': 12, 'misc': 'SpaceAfter=No'}, {'id': 2, 'text': '車', 'lemma': '車', 'upos': 'NOUN', 'xpos': '名詞-普通名詞-一般', 'start_char': 12, 'end_char': 13, 'mis

,id,text,lemma,upos,xpos,start_char,end_char,misc,feats
0,1,この,此の,DET,連体詞,10,12,SpaceAfter=No,NaN
1,2,車,車,NOUN,名詞-普通名詞-一般,12,13,SpaceAfter=No,NaN
2,3,は,は,ADP,助詞-係助詞,13,14,SpaceAfter=No,NaN
3,4,速く,早い,ADJ,形容詞-一般-形容詞,14,16,SpaceAfter=No,NaN
4,5,あり,有る,VERB,動詞-非自立可能-五段-ラ行,16,18,SpaceAfter=No,NaN
5,6,ませ,ます,AUX,助動詞-助動詞-マス,18,20,SpaceAfter=No,NaN
6,7,ん,ず,AUX,助動詞-助動詞-ヌ,20,21,SpaceAfter=No,Polarity=Neg
7,8,でし,です,AUX,助動詞-助動詞-デス,21,23,SpaceAfter=No,NaN
8,9,た,た,AUX,助動詞-助動詞-タ,23,24,SpaceAfter=No,NaN
9,10,。,。,PUNCT,補助記号-句点,24,25,SpaceAfter=No,NaN


                              ROOT                              
                               |                                 
                               S                                
                           ____|_____________________________    
                          S                                  |  
             _____________|____                              |   
            |                  VP                            |  
            |         _________|________                     |   
            |        |                  VP                   |  
            |        |     _____________|___                 |   
            |        |    |                 VP               |  
            |        |    |          _______|_______         |   
            PP       |    |         VP              VP       |  
       _____|____    |    |     ____|___         ___|___     |   
    BASENP       |  ADJP  |    |        VP      |       VP   |  
  ____|_____     |

,id,text,lemma,upos,xpos,start_char,end_char,misc
0,1,怒っ,怒る,VERB,動詞-一般-五段-ラ行,25,27,SpaceAfter=No
1,2,て,て,SCONJ,助詞-接続助詞,27,28,SpaceAfter=No
2,3,しまっ,仕舞う,VERB,動詞-非自立可能-五段-ワア行,28,31,SpaceAfter=No
3,4,た,た,AUX,助動詞-助動詞-タ,31,32,SpaceAfter=No
4,5,。,。,PUNCT,補助記号-句点,32,33,SpaceAfter=No


                   ROOT              
                    |                 
                    S                
                 ___|_____________    
                S                 |  
                |                 |   
                VP                |  
       _________|________         |   
      |                  VP       |  
      |              ____|___     |   
      VP            |        VP   |  
  ____|____         |        |    |   
VERB     SCONJ     VERB     AUX PUNCT
 |         |        |        |    |   
 怒っ        て       しまっ       た    。  

[{'id': 1, 'text': '放り込ん', 'lemma': '放り込む', 'upos': 'VERB', 'xpos': '動詞-一般-五段-マ行', 'start_char': 33, 'end_char': 37, 'misc': 'SpaceAfter=No'}, {'id': 2, 'text': 'で', 'lemma': 'て', 'upos': 'SCONJ', 'xpos': '助詞-接続助詞', 'start_char': 37, 'end_char': 38, 'misc': 'SpaceAfter=No'}, {'id': 3, 'text': 'おい', 'lemma': '於く', 'upos': 'VERB', 'xpos': '動詞-非自立可能-五段-カ行', 'start_char': 38, 'end_char': 40, 'misc': 'SpaceAfter=No'}, {'id'

,id,text,lemma,upos,xpos,start_char,end_char,misc
0,1,放り込ん,放り込む,VERB,動詞-一般-五段-マ行,33,37,SpaceAfter=No
1,2,で,て,SCONJ,助詞-接続助詞,37,38,SpaceAfter=No
2,3,おい,於く,VERB,動詞-非自立可能-五段-カ行,38,40,SpaceAfter=No
3,4,て,て,SCONJ,助詞-接続助詞,40,41,SpaceAfter=No
4,5,くれ,呉れる,VERB,動詞-非自立可能-下一段-ラ行,41,43,SpaceAfter=No
5,6,て,て,SCONJ,助詞-接続助詞,43,44,SpaceAfter=No
6,7,よかっ,良い,ADJ,形容詞-非自立可能-形容詞,44,47,SpaceAfter=No
7,8,た,た,AUX,助動詞-助動詞-タ,47,48,SpaceAfter=No
8,9,。,。,PUNCT,補助記号-句点,48,49,SpaceAfter=No


                                         ROOT               
                                          |                  
                                          S                 
                                __________|______________    
                               S                         |  
                           ____|_______________          |   
                         SBAR                  |         |  
                 _________|__________          |         |   
                S                    |         |         |  
                |                    |         |         |   
                VP                   |         S         |  
       _________|_________           |         |         |   
      |                   VP         |        ADJP       |  
      |               ____|____      |     ____|____     |   
      VP             VP        VP    |    |         VP   |  
  ____|____      ____|____     |     |    |         |    |   
VERB     SCONJ V

# Mecab

In [7]:
import subprocess

result = subprocess.run(['mecab'], input=text, text=True, capture_output=True)
lines = result.stdout.strip().split('\n')
rows = []
for line in lines:
    if line == 'EOS' or not line:
        continue
    cols = line.split('\t')
    if len(cols) < 2:
        continue
    surface = cols[0]
    features = cols[1].split(',')
    row = {
        "surface": surface,
        "pos": features[0] if len(features) > 0 else "",
        "pos_detail1": features[1] if len(features) > 1 else "",
        "pos_detail2": features[2] if len(features) > 2 else "",
        "pos_detail3": features[3] if len(features) > 3 else "",
        "conjugation_form": features[4] if len(features) > 4 else "",
        "conjugation_type": features[5] if len(features) > 5 else "",
        "base_form": features[6] if len(features) > 6 else "",
        "reading": features[7] if len(features) > 7 else "",
        "pronunciation": features[8] if len(features) > 8 else "",
    }
    rows.append(row)

display(pd.DataFrame(rows))
print(tabulate(pd.DataFrame(rows), headers="keys", tablefmt="github", showindex=False))

,surface,pos,pos_detail1,pos_detail2,pos_detail3,conjugation_form,conjugation_type,base_form,reading,pronunciation
0,食べ,動詞,自立,*,*,一段,未然形,食べる,タベ,タベ
1,させ,動詞,接尾,*,*,一段,未然形,させる,サセ,サセ
2,られ,動詞,接尾,*,*,一段,連用形,られる,ラレ,ラレ
3,まし,助動詞,*,*,*,特殊・マス,連用形,ます,マシ,マシ
4,た,助動詞,*,*,*,特殊・タ,基本形,た,タ,タ
5,。,記号,句点,*,*,*,*,。,。,。
6,この,連体詞,*,*,*,*,*,この,コノ,コノ
7,車,名詞,一般,*,*,*,*,車,クルマ,クルマ
8,は,助詞,係助詞,*,*,*,*,は,ハ,ワ
9,速く,形容詞,自立,*,*,形容詞・アウオ段,連用テ接続,速い,ハヤク,ハヤク


| surface   | pos    | pos_detail1   | pos_detail2   | pos_detail3   | conjugation_form   | conjugation_type   | base_form   | reading    | pronunciation   |
|-----------|--------|---------------|---------------|---------------|--------------------|--------------------|-------------|------------|-----------------|
| 食べ      | 動詞   | 自立          | *             | *             | 一段               | 未然形             | 食べる      | タベ       | タベ            |
| させ      | 動詞   | 接尾          | *             | *             | 一段               | 未然形             | させる      | サセ       | サセ            |
| られ      | 動詞   | 接尾          | *             | *             | 一段               | 連用形             | られる      | ラレ       | ラレ            |
| まし      | 助動詞 | *             | *             | *             | 特殊・マス         | 連用形             | ます        | マシ       | マシ            |
| た        | 助動詞 | *             | *             | *             | 特殊・タ           | 基本形             | た          | タ         

# Fugashi

In [12]:
import fugashi
import unidic
tagger = fugashi.Tagger('-d ' + unidic.DICDIR)

In [13]:
tagger.parse(text)
for word in tagger(text):
    print(word, word.feature.lemma, word.pos, sep='\t')

食べ	食べる	動詞,一般,*,*
させ	させる	助動詞,*,*,*
られ	られる	助動詞,*,*,*
まし	ます	助動詞,*,*,*
た	た	助動詞,*,*,*
。	。	補助記号,句点,*,*
この	此の	連体詞,*,*,*
車	車	名詞,普通名詞,一般,*
は	は	助詞,係助詞,*,*
速く	早い	形容詞,一般,*,*
あり	有る	動詞,非自立可能,*,*
ませ	ます	助動詞,*,*,*
ん	ず	助動詞,*,*,*
でし	です	助動詞,*,*,*
た	た	助動詞,*,*,*
。	。	補助記号,句点,*,*
怒っ	怒る	動詞,一般,*,*
て	て	助詞,接続助詞,*,*
しまっ	仕舞う	動詞,非自立可能,*,*
た	た	助動詞,*,*,*
。	。	補助記号,句点,*,*
放り込ん	放り込む	動詞,一般,*,*
で	て	助詞,接続助詞,*,*
おい	置く	動詞,非自立可能,*,*
て	て	助詞,接続助詞,*,*
くれ	呉れる	動詞,非自立可能,*,*
て	て	助詞,接続助詞,*,*
よかっ	良い	形容詞,非自立可能,*,*
た	た	助動詞,*,*,*
。	。	補助記号,句点,*,*


In [14]:
def analyze_verb_conjugation(text):
    words = list(tagger(text))
    analysis = {
        "input": text,
        "dictionary_form": None,
        "conjugation_chain": [],
        "segments": []
    }

    for word in words:
        pos = word.pos
        lemma = word.feature.lemma
        inflection_type = word.feature.cType  # 活用型
        inflection_form = word.feature.cForm  # 活用形

        if '動詞' in pos or '助動詞' in pos:
            # Add to conjugation chain
            segment_info = {
                "surface": word.surface,
                "lemma": lemma,
                "pos": pos,
                "conjugation_type": inflection_type,
                "conjugation_form": inflection_form
            }
            analysis["segments"].append(segment_info)

            if analysis["dictionary_form"] is None:
                analysis["dictionary_form"] = lemma

            if inflection_form and inflection_form != "*":
                analysis["conjugation_chain"].append({
                    "type": inflection_type,
                    "form": inflection_form,
                    "surface": word.surface
                })

    return analysis

# Test examples
test_inputs = [
    "食べさせられました",   # causative passive past polite
    "読まなかった",         # negative past
    "行っている",           # te-form + progressive
    "見られる",             # potential/passive
    "書かせていただきます" # causative + humble polite
]

for text in test_inputs:
    print("=" * 50)
    result = analyze_verb_conjugation(text)
    print(f"Input: {result['input']}")
    print(f"Dictionary form: {result['dictionary_form']}")
    print("Conjugation chain:")
    for c in result["conjugation_chain"]:
        print(f"  - {c['surface']} ({c['type']} - {c['form']})")


Input: 食べさせられました
Dictionary form: 食べる
Conjugation chain:
  - 食べ (下一段-バ行 - 未然形-一般)
  - させ (下一段-サ行 - 未然形-一般)
  - られ (助動詞-レル - 連用形-一般)
  - まし (助動詞-マス - 連用形-一般)
  - た (助動詞-タ - 終止形-一般)
Input: 読まなかった
Dictionary form: 読む
Conjugation chain:
  - 読ま (五段-マ行 - 未然形-一般)
  - なかっ (助動詞-ナイ - 連用形-促音便)
  - た (助動詞-タ - 連体形-一般)
Input: 行っている
Dictionary form: 行く
Conjugation chain:
  - 行っ (五段-カ行 - 連用形-促音便)
  - いる (上一段-ア行 - 終止形-一般)
Input: 見られる
Dictionary form: 見る
Conjugation chain:
  - 見 (上一段-マ行 - 未然形-一般)
  - られる (助動詞-レル - 連体形-一般)
Input: 書かせていただきます
Dictionary form: 書く
Conjugation chain:
  - 書か (五段-カ行 - 未然形-一般)
  - せ (下一段-サ行 - 連用形-一般)
  - いただき (五段-カ行 - 連用形-一般)
  - ます (助動詞-マス - 終止形-一般)


In [15]:
import fugashi
import unidic

tagger = fugashi.Tagger('-d ' + unidic.DICDIR)
text = "食べさせられました"

words = list(tagger(text))

# Base dictionary form and first surface
dictionary_form = words[0].feature.lemma

# Map auxiliaries to grammatical functions
AUX_MEANINGS = {
    'させる': 'causative',
    'られる': 'passive',
    'ます': 'polite',
    'た': 'past',
    'ない': 'negative',
    'たい': 'desire',
    'う': 'volitional',
}

# Start with empty conjugated string
conjugated = ""
chain = []

for i, word in enumerate(words):
    surface = word.surface
    lemma = word.feature.lemma
    cType = word.feature.cType
    cForm = word.feature.cForm

    conjugated += surface

    # Find grammatical function if auxiliary (except main verb)
    gram_func = ""
    if i > 0 and lemma in AUX_MEANINGS:
        gram_func = AUX_MEANINGS[lemma]

    chain.append({
        'conjugated': conjugated,
        'surface': surface,
        'lemma': lemma,
        'cType': cType,
        'cForm': cForm,
        'function': gram_func,
    })

# Output
print(f"Input: {text}")
print(f"Dictionary form: {dictionary_form}")
print("Conjugation chain with cumulative forms:")

for step in chain:
    func_str = f" - {step['function']}" if step['function'] else ""
    print(f"  - {step['conjugated']} ({step['lemma']} - {step['cType']} - {step['cForm']}){func_str}")


Input: 食べさせられました
Dictionary form: 食べる
Conjugation chain with cumulative forms:
  - 食べ (食べる - 下一段-バ行 - 未然形-一般)
  - 食べさせ (させる - 下一段-サ行 - 未然形-一般) - causative
  - 食べさせられ (られる - 助動詞-レル - 連用形-一般) - passive
  - 食べさせられまし (ます - 助動詞-マス - 連用形-一般) - polite
  - 食べさせられました (た - 助動詞-タ - 終止形-一般) - past


# Spacy

In [24]:
!uv run python -m spacy download ja_core_news_sm

/Users/chaosarium/Documents/Repos/Influx/influx_nlp/.venv/bin/python3: No module named pip


In [27]:
import spacy
import pykakasi
japanese_nlp = spacy.load("ja_core_news_sm", disable = ['ner', 'parser'])
# japanese_nlp.add_pipe("custom_sentence_splitter", first=True)
japanese_nlp.add_pipe('sentencizer')
hiraganaConverter = pykakasi.kakasi()

doc = japanese_nlp(text)

In [28]:
for sentenceIndex, sentence in enumerate(doc.sents):
    for token in sentence:
        print("---")
        print(token)
        print(token.lemma_)

        reading = list()
        lemmaReading = list()
        if True or language == 'japanese':
            result = hiraganaConverter.convert(token.text)
            for x in result:
                reading.append(x['hira'])
            
            result = hiraganaConverter.convert(token.lemma_)
            for x in result:
                lemmaReading.append(x['hira'])
        
            reading = ''.join(reading)
            lemmaReading = ''.join(lemmaReading)
            
        print(reading)
        print(lemmaReading)

---
食べ
食べる
たべ
たべる
---
させ
させる
させ
させる
---
られ
られる
られ
られる
---
まし
ます
まし
ます
---
た
た
た
た
